In [1]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time
import matplotlib
import matplotlib.pyplot as plt

import dataset.dataset as dataset
import datasplit.datasplit as datasplit
import model.models as models
import trainer.trainer as trainer
import utils.utils as utils

torch.cuda.device_count()

cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cuda2 = torch.device('cuda:2')
cuda3 = torch.device('cuda:3')

device = torch.device(cuda0 if torch.cuda.is_available() else "cpu")

# INIT

In [2]:
# transforms
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

# dataset
root = '/Volumes/Macintosh HD/DATASETS/GUITAR-FX/Mono_Discrete'
excl_folders = ['MT2']
spectra_folder= 'mel_22050_1024_512'
proc_settings_csv = 'proc_settings.csv'
max_num_settings=3

dataset = dataset.FxDataset(root=root,
                            excl_folders=excl_folders, 
                            spectra_folder=spectra_folder, 
                            processed_settings_csv=proc_settings_csv,
                            max_num_settings=max_num_settings,
                            transform=transform)
dataset.init_dataset()
# dataset.generate_mel()

# split
split = datasplit.DataSplit(dataset, shuffle=True)

# loaders
train_loader, val_loader, test_loader = split.get_split(batch_size=100)

print('dataset size: ', len(dataset))
print('train set size: ', len(split.train_sampler))
print('val set size: ', len(split.val_sampler))
print('test set size: ', len(split.test_sampler))
dataset.fx_to_label

# TRAIN SetNetCond

In [4]:
# model
setnetcond = models.SettingsNetCond(n_settings= dataset.max_num_settings,
                                    mel_shape=dataset.mel_shape, 
                                    num_embeddings=dataset.num_fx, 
                                    embedding_dim=50)
# optimizer
optimizer = optim.Adam(setnetcond.parameters(), lr=0.001)
# loss function
loss_func = nn.MSELoss(reduction='mean')

print(setnetcond)

In [6]:
# SAVE
models_folder = '../../saved/models'
model_name = '20201017_setnetcond_mono_disc_best'
results_folder = '../../saved/results'
results_subfolder = '20201017_setnetcond_mono_disc'

In [7]:
# TRAIN and TEST SettingsNetCond OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses, all_val_losses, all_test_losses = [],[],[]
all_train_correct, all_val_correct, all_test_correct = [],[],[]
all_train_results, all_val_results, all_test_results = [],[],[]

best_val_correct = 0
early_stop_counter = 0

start = time.time()

for epoch in range(100):
    train_loss, train_correct, train_results = trainer.train_settings_cond_net(
        model=setnetcond,
        optimizer=optimizer, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch,
        loss_function=loss_func, 
        device=device
    )
    
    val_loss, val_correct, val_results = trainer.val_settings_cond_net(
        model=setnetcond, 
        val_loader=val_loader, 
        val_sampler=split.val_sampler,
        loss_function=loss_func, 
        device='cpu'
    )
    
    test_loss, test_correct, test_results = trainer.test_settings_cond_net(
        model=setnetcond, 
        test_loader=test_loader, 
        test_sampler=split.test_sampler,
        loss_function=loss_func, 
        device='cpu'
    )
    # save model
    if val_correct > best_val_correct:
        best_val_correct = val_correct
        torch.save(setnetcond, '%s/%s' % (models_folder, model_name))
        early_stop_counter = 0
        print('\n=== saved best model ===\n')
    else:
        early_stop_counter += 1
        
    # append results
    all_train_losses.append(train_loss)
    all_val_losses.append(val_loss)
    all_test_losses.append(test_loss)
    
    all_train_correct.append(train_correct)
    all_val_correct.append(val_correct)
    all_test_correct.append(test_correct)
    
    all_train_results.append(train_results)
    all_val_results.append(val_results)
    all_test_results.append(test_results)

    if early_stop_counter == 15:
        print('\n--- early stop ---\n')
        break

stop = time.time()
print(f"Training time: {stop - start}s")

Train Epoch: 0	[5000/88956 (6%)]	Total Loss: 7.6193	Avg Loss: 0.0015
Train Epoch: 0	[10000/88956 (11%)]	Total Loss: 11.1931	Avg Loss: 0.0011
Train Epoch: 0	[15000/88956 (17%)]	Total Loss: 14.4323	Avg Loss: 0.0010
Train Epoch: 0	[20000/88956 (22%)]	Total Loss: 17.3874	Avg Loss: 0.0009
Train Epoch: 0	[25000/88956 (28%)]	Total Loss: 19.8195	Avg Loss: 0.0008
Train Epoch: 0	[30000/88956 (34%)]	Total Loss: 21.9337	Avg Loss: 0.0007
Train Epoch: 0	[35000/88956 (39%)]	Total Loss: 23.8249	Avg Loss: 0.0007
Train Epoch: 0	[40000/88956 (45%)]	Total Loss: 25.5354	Avg Loss: 0.0006
Train Epoch: 0	[45000/88956 (51%)]	Total Loss: 26.9943	Avg Loss: 0.0006
Train Epoch: 0	[50000/88956 (56%)]	Total Loss: 28.3639	Avg Loss: 0.0006
Train Epoch: 0	[55000/88956 (62%)]	Total Loss: 29.6332	Avg Loss: 0.0005
Train Epoch: 0	[60000/88956 (67%)]	Total Loss: 30.8635	Avg Loss: 0.0005
Train Epoch: 0	[65000/88956 (73%)]	Total Loss: 31.9583	Avg Loss: 0.0005
Train Epoch: 0	[70000/88956 (79%)]	Total Loss: 33.0985	Avg Loss: 0.

In [8]:
# BEST RESULTS
print('Accuracy: ', 100 * max(all_train_correct) / train_set_size)
print('Epoch: ', np.argmax(all_train_correct))
print()
print('Accuracy: ', 100 * max(all_val_correct) / val_set_size)
print('Epoch: ', np.argmax(all_val_correct))
print()
print('Accuracy: ', 100 * max(all_test_correct) / test_set_size)
print('Epoch: ', np.argmax(all_test_correct))
print()

Accuracy:  92.74697603309501
Epoch:  95

Accuracy:  80.09104704097116
Epoch:  95

Accuracy:  80.05746428715956
Epoch:  95



In [9]:
# SAVE RESULTS - all losses, all correct, best results
all_train_losses_npy = np.array(all_train_losses)
all_train_correct_npy = np.array(all_train_correct)
best_train_results_npy = np.array(all_train_results[95])

all_val_losses_npy = np.array(all_val_losses)
all_val_correct_npy = np.array(all_val_correct)
best_val_results_npy = np.array(all_val_results[95])

all_test_losses_npy = np.array(all_test_losses)
all_test_correct_npy = np.array(all_test_correct)
best_test_results_npy = np.array(all_test_results[95])

fx_labels_npy = np.array(list(dataset.fx_to_label.keys()))

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_losses')), arr=all_train_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_correct')), arr=all_train_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_train_results')), arr=best_train_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_losses')), arr=all_val_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_correct')), arr=all_val_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_val_results')), arr=best_val_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_losses')), arr=all_test_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_correct')), arr=all_test_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_test_results')), arr=best_test_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'fx_labels')), arr=fx_labels_npy)